In [1]:
## Importing the required libraries

import json 
import string
import pandas as pd
import numpy as np

with open('recipes.json') as f:
    info = json.load(f)
with open('recipes.json') as f:
    info1 = json.load(f)

fd = pd.DataFrame( None,columns =['line','title','categories','ingredients','directions']) 

In [2]:
## Creating a function for the pre-processing of the query

def check_query(query):
    if isinstance(query,list) == True:
        query = ' '.join(query)
    query = query.lower() 
    tab = query.maketrans(string.punctuation,'                                ') 
    query = query.translate(tab) 
   # '01234567890123456789012345678901'
    tab = query.maketrans(string.digits,'          ') 
    query = query.translate(tab) 

    query = query.split()
    if isinstance(query,list) == True:
        i = 0
        while i < len(query):
            if len(query[i]) < 3:
                query.remove(query[i])
                i -= 1
            i += 1
    
    return query  

In [3]:
## Separating the words as per the keys

for rec in range(len(info)):
    w_list = []
    
    for head in ['title','categories','ingredients','directions']:
        try:
            info[rec][head] = check_query(info[rec][head]) 
            
            wd, freq = np.unique(info[rec][head],return_counts= True)
            wd = list(wd)
            w_list = w_list+wd
            
            for w in range(len(wd)):
                if wd[w] in fd.index:
                    if isinstance(fd[head][wd[w]],list) == False:
                        fd[head][wd[w]] = [rec]
                    else:
                        fd[head][wd[w]].append(rec)
                else:
                    fd.loc[wd[w]] = {'line':np.nan,'title':np.nan,'categories':np.nan,'ingredients':np.nan,'directions':np.nan}
                    fd[head][wd[w]] = [rec]  
        except:
            pass
    wds,freq = np.unique(w_list,return_counts= True)
    wds = list(wds)
    
    for w in range(len(wds)):
        if isinstance(fd['line'][wds[w]],list) == False:
            fd['line'][wds[w]] = [rec]
        else:
            fd['line'][wds[w]].append(rec)
                    
fd              

,line,title,categories,ingredients,directions
adult,"[0, 11966, 14662, 15721, 17620]",[0],NaN,NaN,"[11966, 14662, 15721, 17620]"
cheese,"[0, 6, 28, 32, 34, 42, 54, 55, 91, 92, 94, 106...","[0, 32, 34, 92, 139, 159, 166, 176, 197, 251, ...","[0, 6, 28, 32, 34, 42, 54, 91, 92, 133, 139, 1...","[6, 28, 34, 42, 91, 92, 94, 106, 133, 139, 145...","[0, 28, 32, 34, 42, 55, 94, 106, 133, 139, 145..."
pimiento,"[0, 117, 503, 856, 899, 1195, 1204, 1422, 1912...","[0, 503, 6628, 7146, 11330, 11331, 11332, 11333]",NaN,"[117, 503, 856, 899, 1195, 1204, 1422, 1912, 2...","[117, 503, 6628, 7146, 7545, 8284, 11330, 1133..."
alabama,"[0, 10317, 17320]",NaN,"[0, 10317, 17320]",NaN,NaN
cheddar,"[0, 28, 30, 248, 252, 254, 267, 286, 365, 456,...","[252, 267, 286, 365, 772, 798, 804, 805, 856, ...","[0, 248, 254, 267, 286, 772, 773, 788, 798, 80...","[0, 28, 30, 248, 252, 254, 267, 286, 365, 456,...","[0, 30, 252, 254, 786, 788, 798, 805, 856, 895..."
...,...,...,...,...,...
kefalotíri,[17719],NaN,NaN,[17719],NaN
mozarella,[17723],[17723],NaN,NaN,NaN
belt,[17730],NaN,NaN,NaN,[17730]
rubs,[17730],NaN,NaN,NaN,[17730]


In [6]:
## Creating a search function 

def search(query, ordering = 'normal', count = 10):
    query = check_query(query) 

    try:
        a= set(fd['line'][query[0]])
    except:
        return
    
    for k in query:
        if k not in fd.index:

            return 
        else:
            a = a & set(fd['line'][k])

    sc_r = []
  
    if  ordering ==  'normal' :
        for k in a:
            opt = 0.0
            
            try:
                for p in range(len(info[k]['title'])):
                    if info[k]['title'][p] in query:
                        opt += 8.0
            except:
                opt += 0.0
            
            try:
                for q in range(len(info[k]['categories'])):
                    if info[k]['categories'][q] in query:
                        opt += 4.0
            except:
                opt += 0.0
            
            try:
                for r in range(len(info[k]['ingredients'])):
                    if info[k]['ingredients'][r] in query:
                        opt += 2.0
            except:
                opt += 0.0
            
            try:
                for s in range(len(info[k]['directions'])):
                    if info[k]['directions'][s] in query:
                        opt += 1.0
            except:
                opt += 0.0
                          
            try:
                opt += float(info[k]['rating'])
            except:
                opt += 0.0
            sc_r.append((opt,k))
            
        if len(sc_r) < count:
            count = len(sc_r) 
        for i in range(count):
            c = max(sc_r)
            print(info1[c[1]]['title'])
            sc_r.remove(c)
                                      
    if  ordering ==  'simple' :
        for k in a:  
            try:
                
                p =len(info1[k]['ingredients'])
                q =len(info1[k]['directions'])
                if p >1 and q >1:
                    opt = p*q
                    sc_r.append((opt,k))   
            except:
                pass       
        if len(sc_r) < count:
              count = len(sc_r) 
        for i in range(count):
            c = min(sc_r)
            print(info1[c[1]]['title'])
            sc_r.remove(c)    
            
    if  ordering ==  'healthy' :
        for k in a:
            try:
                opt = []
                for n in range(1,8):                    
                    opt1 = (abs(info1[k]['calories']-(n*510))/510) + (2*abs(info1[k]['protein']-(n*18))/18) + (4*abs(info1[k]['fat']-(n*150))/150)
                    
       
                    opt.append(opt1)
                opt2 = min(opt)

                sc_r.append((opt2,k))
                
            except:
                pass   
        if len(sc_r) < count:
              count = len(sc_r) 
        for i in range(count):
            c = min(sc_r)
            print(info1[c[1]]['title'])
            sc_r.remove(c)        

In [7]:
## Example to check the search function

search('coffee', ordering = 'healthy', count = 6)

Tortillas in Black-Bean Purée 
Vegetable Medley in Garlic-Chile Sauce 
Fudgy Peanut Brownies 
Grilled Lemongrass Lamb Chops with Herbs 
Mocha Cake with Malted Semifreddo 
Cappuccino Brownies with White Chocolate-Espresso Sauce 
